In [34]:
import pandas as pd
import numpy as np

import warnings
from ast import literal_eval
warnings.filterwarnings("ignore")

In [35]:
def extract_first_number(text: str):
    import re

    # getting numbers from string
    temp = re.findall(r'\d+', text)
    res = list(map(int, temp))

    if len(res):
        return res[0]

In [36]:
def clean_phone_number(phone_number: str):
    phone_number = phone_number.strip(' ')
    phone_number = phone_number.replace('x', '*')
    phone_number = phone_number.replace(' ', '')

    return phone_number

In [37]:
def extract_apartment_type(entities):

    apart_type_mapping = {'c.c.m.n': 'ccmn', 'mini': 'mini', 'ccmn': 'ccmn', 'cao_cấp': 'cao_cấp',
                          'mi ni': 'mini', 'ch': 'chung_cư',
                          'chung_cư':'chung_cư', 'căn': 'chung_cư', 'căn_hộ': 'chung_cư',
                          'penhouse': 'penhouse', 'sang_trọng': 'cao_cấp',
                          'studio': 'studio', 'tập_thể': 'tập_thể', 'nhà_tập_thể': 'tập_thể','tt': 'tập_thể'}

    entities_list = [entity[1] for entity in entities]
    apart_idx = list(np.where(np.array(entities_list) == 'APART')[0])
    cleaned_entities = []

    for idx in apart_idx:
        apart_type = entities[idx][0]
        res = []
        flag = False
        for key in apart_type_mapping.keys():
            if key in apart_type:
                res.append(apart_type_mapping[key])
                flag = True

        if flag:
            res = list(set(res))
            cleaned_entities.append(' '.join(res))
        # else:
        #    cleaned_entities.append(entities_list[idx][0])

    return cleaned_entities

In [38]:
def extract_room(entities):
    room_mapping = {'phòng ngủ': 'phòng_ngủ', 'pn': 'phòng_ngủ', 'phòng ng': 'phòng_ngủ',
                    'ngủ': 'phòng_ngủ', 'khách': 'phòng_khách',
                    'phòng khách': 'phòng_khách', 'pk': 'phòng_khách',
                    'wc': 'vệ_sinh', 'vsinh': 'vệ_sinh', 'công_trình_phụ': 'vệ_sinh',
                    'vệ': 'vệ_sinh', 'nhà_vệ_sinh': 'vệ_sinh', 'tắm': 'vệ_sinh', 'w': 'vệ_sinh',
                    'vs': 'vệ_sinh'}

    entities_list = [entity[1] for entity in entities]
    room_idx = list(np.where(np.array(entities_list) == 'ROOM')[0])
    room_entities = []

    for idx in room_idx:
        room = entities[idx][0]

        temp_result = []
        for key in room_mapping.keys():
            flag = False
            if key in room:
                start_idx = room.find(key)
                try:
                    for res in temp_result:
                        if room_mapping[key] in res:
                            flag = True
                    if flag:
                        continue

                    if room[start_idx-1] == ',':
                        temp_result.append('1 {}'.format(room_mapping[key]))
                    elif room[start_idx - 1] == ' ' and room[start_idx-2].isdigit():
                        temp_result.append('{} {}'.format(room[start_idx-2], room_mapping[key]))
                    elif room[start_idx - 1].isdigit():
                        temp_result.append('{} {}'.format(room[start_idx - 1], room_mapping[key]))
                    else:
                        temp_result.append('1 {}'.format(room_mapping[key]))
                except:
                    temp_result.append('1 {}'.format(room_mapping[key]))
        temp_result = list(set(temp_result))

        room_entities.extend(temp_result)

    return room_entities

In [39]:
def extract_furniture_type(entities):

    fur_type_mapping = {'đầy_đủ': 'full', 'đủ đồ': 'full', 'full': 'full',
                        'cơ_bản': 'cơ_bản', 'nguyên_bản': 'nguyên_bản',
                        'cao_cấp': 'cao_cấp'}

    entities_list = [entity[1] for entity in entities]
    fur_type_idx = list(np.where(np.array(entities_list) == 'FUR-TYPE')[0])
    cleaned_entities = []

    for idx in fur_type_idx:
        fur_type = entities[idx][0]
        res = []
        flag = False
        for key in fur_type_mapping.keys():
            if key in fur_type:
                res.append(fur_type_mapping[key])
                flag = True

        if flag:
            res = list(set(res))
            cleaned_entities.append(' '.join(res))
        # else:
        #    cleaned_entities.append(entities_list[idx][0])

    return cleaned_entities

In [40]:
def fill_empty_data(converted_df):
    columns = converted_df.columns
    data = []

    for i in range(len(converted_df)):
        entities = literal_eval(converted_df.iloc[i]['ENTITIES'])
        news = converted_df.iloc[i]

        entities_list = [entity[1] for entity in entities]
        # area_idx = list(np.where(np.array(entities_list) == 'AREA')[0])

        def find_first_entity(entities_list, entity_label):
            for i in range(len(entities_list)):
                if  entity_label in entities_list[i]:
                    return i

            return -1

        room_entities = extract_room(entities)
        apartment_entities = extract_apartment_type(entities)
        fur_type_entities = extract_furniture_type(entities)

        if pd.isna(news['LOCATION']):
            loc_id = find_first_entity(entities_list, 'LOC')
            if loc_id != -1:
                news['LOCATION'] = entities[loc_id]

        if pd.isna(news['PRICE']):
            price_id = find_first_entity(entities_list, 'PRICE')
            if price_id != -1:
                news['PRICE'] = entities[price_id]

        if pd.isna(news['AREA']):
            area_id = find_first_entity(entities_list, 'AREA')
            if area_id != -1:
                news['PRICE'] = entities[area_id]

        if pd.isna(news['BATH_ROOM_NUMBER']):
            room_id = find_first_entity(room_entities, 'vệ_sinh')
            if room_id != -1:
                try:
                    news['BATH_ROOM_NUMBER'] = int(room_entities[room_id].replace(' vệ_sinh', ''))
                except:
                    pass

        if pd.isna(news['LIVING_ROOM']):
            room_id = find_first_entity(room_entities, 'phòng_khách')
            if room_id != -1:
                try:
                    news['LIVING_ROOM'] = int(room_entities[room_id].replace(' phòng_khách', ''))
                except:
                    pass

        if pd.isna(news['BEDROOM_NUMBER']):
            room_id = find_first_entity(room_entities, 'phòng_ngủ')
            if room_id != -1:
                try:
                    news['BEDROOM_NUMBER'] = int(room_entities[room_id].replace(' phòng_ngủ', ''))
                except:
                    pass

        if len(apartment_entities):
            news['APARTMENT_TYPE'] = apartment_entities[0]

        if pd.isna(news['PROJECT']) or ('đang cập nhật' == news['PROJECT']):
            project_id = find_first_entity(entities_list, 'PJ')
            if project_id != -1:
                news['PROJECT'] = entities[project_id][0]

        if pd.isna(news['FURNITURE_TYPE']) and len(fur_type_entities):
            news['FURNITURE_TYPE'] = fur_type_entities[0]

        if pd.isna(news['CONVENIENT']):
            conv_id = list(np.where(np.array(entities_list) == 'CONV')[0])
            conv_entities = [entities[i][0] for i in conv_id]

            cleaned_convenients = []
            for entity in conv_entities:
                entity = entity.strip(', ')
                words = entity.split(',')
                words = [word for word in words if word not in ['', ' ', ',']]
                cleaned_convenients.extend(words)
            news['CONVENIENT'] = cleaned_convenients

        if pd.isna(news['FURNITURE']):
            fur_id = list(np.where(np.array(entities_list) == 'FUR')[0])
            fur_entities = [entities[i][0] for i in fur_id]

            cleaned_furnitures = []
            for entity in fur_entities:
                entity = entity.strip(', ')
                words = entity.split(',')
                words = [word for word in words if word not in ['', ' ', ',']]
                cleaned_furnitures.extend(words)
            news['FURNITURE'] = cleaned_furnitures

        data.append(news.values)

    return pd.DataFrame(data=data, columns=columns)

In [ ]:
chotot_file = 'chotot.csv'
chotot_df = pd.read_csv(chotot_file)

In [38]:
chotot_df['BATH_ROOM_NUMBER'] = chotot_df['BATH_ROOM_NUMBER'].map(extract_first_number, na_action='ignore')
chotot_df['BEDROOM_NUMBER'] = chotot_df['BEDROOM_NUMBER'].map(extract_first_number, na_action='ignore')
chotot_df['PHONE_NUMBER'] = chotot_df['PHONE_NUMBER'].map(clean_phone_number, na_action='ignore')

True

In [ ]:
filled_df = fill_empty_data(converted_df=chotot_df)

In [8]:
alonhadat_file = 'alonhadat.csv'
alonhadat_df = pd.read_csv(alonhadat_file)

In [10]:
alonhadat_df['PHONE_NUMBER'] = alonhadat_df['PHONE_NUMBER'].map(clean_phone_number, na_action='ignore')

In [11]:
alonhadat_df

,Unnamed: 0,LOCATION,PHONE_NUMBER,NEWS_TYPE,BEDROOM_NUMBER,BATH_ROOM_NUMBER,LIVING_ROOM,AREA,APARTMENT_TYPE,PROJECT,INVESTOR,FLOOR,FURNITURE_TYPE,FURNITURE,CONVENIENT,PRICE,URL,RAW_DESCRIPTION,ENTITIES
0,0,"đường lĩnh nam, phường lĩnh nam, quận hoàng m...",0961632***,NaN,3.0,NaN,NaN,96.0,NaN,căn hộ chung cư new horizion city,NaN,20.0,NaN,NaN,NaN,10.0,https://alonhadat.com.vn/cho-thue-can-ho-chung...,"cho thuê căn hộ chung cư 87 lĩnh nam , hoàng m...","[['<s>', 'FUR-TYPE'], ['căn_hộ chung_cư', 'APA..."
1,1,"đường nguyễn văn huyên, phường bưởi, quận tây...",0961632***,NaN,3.0,NaN,NaN,103.0,NaN,căn hộ chung cư ecolife tây hồ,NaN,14.0,NaN,NaN,NaN,10.0,https://alonhadat.com.vn/cho-thue-can-ho-chung...,cho thuê căn hộ chung cư ecolife tây hồ đường ...,"[['căn_hộ', 'APART'], ['chung_cư ecolife', 'PJ..."
2,2,"đường trường chinh, phường phương mai, quận t...",0961632***,NaN,3.0,NaN,NaN,110.0,NaN,căn hộ chung cư tân hồng hà tower,NaN,6.0,NaN,NaN,NaN,12.5,https://alonhadat.com.vn/cho-thue-can-ho-chung...,"cho thuê căn hộ chung cư 317 trường chinh ,nga...","[['<s>', 'FUR-TYPE'], ['căn_hộ chung_cư', 'APA..."
3,3,"phố liễu giai, phường cống vị, quận ba đình, ...",0978996***,NaN,2.0,NaN,NaN,74.0,NaN,căn hộ chung cư x1-26 liễu giai (liễu giai tower),NaN,NaN,NaN,NaN,NaN,12.0,https://alonhadat.com.vn/chinh-chu-cho-thue-ca...,chính chủ có căn hộ thuộc chung cư liễu giai t...,"[['chung_cư', 'PJ'], ['liễu giai tower - 26 li..."
4,4,"cầu thang số 4 - ct1 đường trần văn lai, phườ...",0981917***,NaN,3.0,NaN,NaN,105.0,NaN,căn hộ chung cư the manor,NaN,1.0,NaN,NaN,NaN,12.5,https://alonhadat.com.vn/chinh-chu-cho-thue-ch...,"chính chủ cho thuê chung cư 105m2 với 3pn, 2vs...","[['chung_cư', 'APART'], ['105m2', 'AREA'], ['3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6471,6471,"đường minh khai, phường minh khai, quận hai b...",0911695***,NaN,2.0,NaN,NaN,80.0,NaN,căn hộ chung cư lilama 124 minh khai,NaN,12.0,NaN,NaN,NaN,8.0,https://alonhadat.com.vn/cho-thue-can-ho-chung...,"cho thuê căn hộ chung cư tại toà lilama, minh ...","[['căn_hộ chung_cư', 'APART'], ['toà lilama, m..."
6472,6472,"an lạc đường lưu hữu phước, phường cầu diễn, ...",0986170***,NaN,2.0,NaN,NaN,80.0,NaN,căn hộ chung cư an lạc,NaN,NaN,NaN,NaN,NaN,7.0,https://alonhadat.com.vn/cho-thue-can-ho-chung...,"cho thuê gấp căn hộ chung cư tòa nhà an lạc , ...","[['căn_hộ chung_cư', 'APART'], ['toà nhà an_lạ..."
6473,6473,"đường việt hưng, phường việt hưng, quận long ...",0345243***,NaN,2.0,NaN,NaN,80.0,NaN,NaN,NaN,15.0,NaN,NaN,NaN,8.0,https://alonhadat.com.vn/cho-thue-can-ho-chung...,"cho thuê căn hộ chung cư happy star việt hưng,...","[['căn_hộ chung_cư', 'APART'], ['2w', 'ROOM'],..."
6474,6474,"đường lê trực, phường điện biên, quận ba đình...",0327582***,NaN,2.0,NaN,NaN,85.0,NaN,khu phức hợp discovery central (discovery comp...,NaN,NaN,NaN,NaN,NaN,25.0,https://alonhadat.com.vn/cho-thue-can-ho-o-dis...,- cho thuê căn hộ ở discovery complex - 8b lê ...,"[['căn_hộ', 'APART'], ['discovery complex', 'P..."


In [12]:
filled_alonhadat_df = fill_empty_data(converted_df=alonhadat_df)

In [13]:
filled_alonhadat_df

,Unnamed: 0,LOCATION,PHONE_NUMBER,NEWS_TYPE,BEDROOM_NUMBER,BATH_ROOM_NUMBER,LIVING_ROOM,AREA,APARTMENT_TYPE,PROJECT,INVESTOR,FLOOR,FURNITURE_TYPE,FURNITURE,CONVENIENT,PRICE,URL,RAW_DESCRIPTION,ENTITIES
0,0,"đường lĩnh nam, phường lĩnh nam, quận hoàng m...",0961632***,NaN,3.0,2.0,NaN,96.0,chung_cư,căn hộ chung cư new horizion city,NaN,20.0,full,[],[],10.0,https://alonhadat.com.vn/cho-thue-can-ho-chung...,"cho thuê căn hộ chung cư 87 lĩnh nam , hoàng m...","[['<s>', 'FUR-TYPE'], ['căn_hộ chung_cư', 'APA..."
1,1,"đường nguyễn văn huyên, phường bưởi, quận tây...",0961632***,NaN,3.0,2.0,1.0,103.0,chung_cư,căn hộ chung cư ecolife tây hồ,NaN,14.0,full,"[tủ bếp, 4 điều_hoà, 2 nóng lạnh, giường, ...",[],10.0,https://alonhadat.com.vn/cho-thue-can-ho-chung...,cho thuê căn hộ chung cư ecolife tây hồ đường ...,"[['căn_hộ', 'APART'], ['chung_cư ecolife', 'PJ..."
2,2,"đường trường chinh, phường phương mai, quận t...",0961632***,NaN,3.0,2.0,NaN,110.0,chung_cư,căn hộ chung cư tân hồng hà tower,NaN,6.0,cơ_bản,[],[],12.5,https://alonhadat.com.vn/cho-thue-can-ho-chung...,"cho thuê căn hộ chung cư 317 trường chinh ,nga...","[['<s>', 'FUR-TYPE'], ['căn_hộ chung_cư', 'APA..."
3,3,"phố liễu giai, phường cống vị, quận ba đình, ...",0978996***,NaN,2.0,2.0,NaN,74.0,NaN,căn hộ chung cư x1-26 liễu giai (liễu giai tower),NaN,NaN,cơ_bản,"[điều_hoà, nóng lạnh, hệ_thống bếp, tủ bếp dư...",[tiện_lợi],12.0,https://alonhadat.com.vn/chinh-chu-cho-thue-ca...,chính chủ có căn hộ thuộc chung cư liễu giai t...,"[['chung_cư', 'PJ'], ['liễu giai tower - 26 li..."
4,4,"cầu thang số 4 - ct1 đường trần văn lai, phườ...",0981917***,NaN,3.0,2.0,1.0,105.0,chung_cư,căn hộ chung cư the manor,NaN,1.0,full,[điều_hoà 2 chiều],"[hiệu thuốhọ, trung_tâm hội_nghị, khu liên_hiệ...",12.5,https://alonhadat.com.vn/chinh-chu-cho-thue-ch...,"chính chủ cho thuê chung cư 105m2 với 3pn, 2vs...","[['chung_cư', 'APART'], ['105m2', 'AREA'], ['3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6471,6471,"đường minh khai, phường minh khai, quận hai b...",0911695***,NaN,2.0,1.0,1.0,80.0,chung_cư,căn hộ chung cư lilama 124 minh khai,NaN,12.0,NaN,[],"[trung_tâm, thuận_tiện]",8.0,https://alonhadat.com.vn/cho-thue-can-ho-chung...,"cho thuê căn hộ chung cư tại toà lilama, minh ...","[['căn_hộ chung_cư', 'APART'], ['toà lilama, m..."
6472,6472,"an lạc đường lưu hữu phước, phường cầu diễn, ...",0986170***,NaN,2.0,1.0,1.0,80.0,chung_cư,căn hộ chung cư an lạc,NaN,NaN,full,[sàn gỗ],[],7.0,https://alonhadat.com.vn/cho-thue-can-ho-chung...,"cho thuê gấp căn hộ chung cư tòa nhà an lạc , ...","[['căn_hộ chung_cư', 'APART'], ['toà nhà an_lạ..."
6473,6473,"đường việt hưng, phường việt hưng, quận long ...",0345243***,NaN,2.0,2.0,NaN,80.0,chung_cư,chung_cư happy star trung_tâm khu đô_thị việt ...,NaN,15.0,NaN,"[nóng lạbế, giường tủă, máy lọc nướ]",[],8.0,https://alonhadat.com.vn/cho-thue-can-ho-chung...,"cho thuê căn hộ chung cư happy star việt hưng,...","[['căn_hộ chung_cư', 'APART'], ['2w', 'ROOM'],..."
6474,6474,"đường lê trực, phường điện biên, quận ba đình...",0327582***,NaN,2.0,NaN,NaN,85.0,chung_cư,khu phức hợp discovery central (discovery comp...,NaN,NaN,cơ_bản,"[điều_hoà, nóng lạnh, tủ âm tường, kệ tủ bế...",[],25.0,https://alonhadat.com.vn/cho-thue-can-ho-o-dis...,- cho thuê căn hộ ở discovery complex - 8b lê ...,"[['căn_hộ', 'APART'], ['discovery complex', 'P..."


In [14]:
filled_alonhadat_df.to_csv('filled_alonhadat.csv')

In [23]:
bds_file = 'bds.csv'
bds_df = pd.read_csv(bds_file)

In [24]:
bds_df

,Unnamed: 0,LOCATION,PHONE_NUMBER,NEWS_TYPE,BEDROOM_NUMBER,BATH_ROOM_NUMBER,LIVING_ROOM,AREA,APARTMENT_TYPE,PROJECT,INVESTOR,FLOOR,FURNITURE_TYPE,FURNITURE,CONVENIENT,PRICE,URL,RAW_DESCRIPTION,ENTITIES
0,0,"dự án vinhomes smart city, đường đại lộ thăng ...",0984521346,NaN,1,NaN,NaN,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,"* cam kết:\n- giá rẻ nhất, dịch vụ tốt nhất.\n...","[['đầy_đủ', 'FUR-TYPE'], ['cơ_bản', 'FUR-TYPE'..."
1,1,"dự án vinhomes smart city đại mỗ, phường tây m...",0933722886,NaN,2,NaN,NaN,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,với kinh nghiệm 5 năm cho thuê nhà và đội ngũ ...,"[['<s>', 'AREA'], ['</s>', 'AREA'], ['<s>', 'A..."
2,2,"dự án vinhomes smart city đại mỗ, phường tây m...",0833451111,NaN,1,NaN,NaN,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,"với sự uy tín, chu đáo và nhiệt tình của đội n...","[['cc vinhomes smart city', 'PJ'], ['1pn', 'RO..."
3,3,"ngõ 27, đường cát linh, phường cát linh, đống ...",0902469188,NaN,3,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,căn hộ đầu hồi 100m2 tầng 3 - tại ngõ 27 cát l...,"[['100m2', 'AREA'], ['ngõ 27 cát linh - đống_đ..."
4,4,"dự án vinhomes d'capitale, đường trần duy hưng...",0944010255,NaN,1,NaN,NaN,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,lh xem thực tế căn hộ 24/7:\n0944010***\n.\n\n...,"[['full', 'FUR-TYPE'], ['tv, giường, tủ, máy_g..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4033,4033,"dự án sunshine city, đường phạm văn đồng, phườ...",0981959535,NaN,2,NaN,NaN,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,chúng tôi chuyên cho thuê các căn hộ tại chung...,"[['sunshine cbắc từ_l', 'PJ'], ['2,3,4,5 phòng..."
4034,4034,"dự án the zei mỹ đình, đường lê đức thọ, phườn...",0348983388,NaN,3,NaN,NaN,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,bên em đang quản lý nhiều căn hộ tại dự án the...,"[['căn_hộ', 'ID'], ['2n m2', 'ROOM'], ['cơ_bản..."
4035,4035,"dự án goldmark city, đường hồ tùng mậu, phường...",0982848648,NaN,2,NaN,NaN,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,do không có nhu cầu sử dụng vợ chồng tôi cần c...,"[['căn_hộ chung_cư cao_cấp', 'APART'], ['toà c..."
4036,4036,"dự án ciputra hà nội, đường lạc long quân, phư...",0976944818,NaN,3,NaN,NaN,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,hiện tại tôi có căn hộ tại tòa the link trong ...,"[['toà the link trong khu đô_thị ciputra.', 'P..."


In [30]:
def extract_project(text: str):
    if 'dự án' in text and ',' in text:
        return text.split(',')[0]

def remove_project(text: str):
    if 'dự án' in text and ',' in text:
        return ','.join(text.split(',')[1:])
    else:
        return text

def clean_bed_room(text):
    if text.isdigit():
        return int(text)

In [26]:
bds_df.iloc[5]['BEDROOM_NUMBER'].isdigit()

False

In [27]:
bds_df['PROJECT'] = bds_df['LOCATION'].map(extract_project, na_action='ignore')

In [28]:
bds_df['LOCATION'] = bds_df['LOCATION'].map(remove_project, na_action='ignore')

In [31]:
bds_df['BEDROOM_NUMBER'] = bds_df['BEDROOM_NUMBER'].map(clean_bed_room, na_action='ignore')

In [32]:
bds_df

,Unnamed: 0,LOCATION,PHONE_NUMBER,NEWS_TYPE,BEDROOM_NUMBER,BATH_ROOM_NUMBER,LIVING_ROOM,AREA,APARTMENT_TYPE,PROJECT,INVESTOR,FLOOR,FURNITURE_TYPE,FURNITURE,CONVENIENT,PRICE,URL,RAW_DESCRIPTION,ENTITIES
0,0,"đường đại lộ thăng long, phường đại mỗ, nam t...",0984521346,NaN,1.0,NaN,NaN,35,NaN,dự án vinhomes smart city,NaN,NaN,NaN,NaN,NaN,4.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,"* cam kết:\n- giá rẻ nhất, dịch vụ tốt nhất.\n...","[['đầy_đủ', 'FUR-TYPE'], ['cơ_bản', 'FUR-TYPE'..."
1,1,"phường tây mỗ, nam từ liêm, hà nội",0933722886,NaN,2.0,NaN,NaN,70,NaN,dự án vinhomes smart city đại mỗ,NaN,NaN,NaN,NaN,NaN,8.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,với kinh nghiệm 5 năm cho thuê nhà và đội ngũ ...,"[['<s>', 'AREA'], ['</s>', 'AREA'], ['<s>', 'A..."
2,2,"phường tây mỗ, nam từ liêm, hà nội",0833451111,NaN,1.0,NaN,NaN,35,NaN,dự án vinhomes smart city đại mỗ,NaN,NaN,NaN,NaN,NaN,4.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,"với sự uy tín, chu đáo và nhiệt tình của đội n...","[['cc vinhomes smart city', 'PJ'], ['1pn', 'RO..."
3,3,"ngõ 27, đường cát linh, phường cát linh, đống ...",0902469188,NaN,3.0,NaN,NaN,100,NaN,None,NaN,NaN,NaN,NaN,NaN,12.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,căn hộ đầu hồi 100m2 tầng 3 - tại ngõ 27 cát l...,"[['100m2', 'AREA'], ['ngõ 27 cát linh - đống_đ..."
4,4,"đường trần duy hưng, phường trung hòa, cầu gi...",0944010255,NaN,1.0,NaN,NaN,38,NaN,dự án vinhomes d'capitale,NaN,NaN,NaN,NaN,NaN,8.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,lh xem thực tế căn hộ 24/7:\n0944010***\n.\n\n...,"[['full', 'FUR-TYPE'], ['tv, giường, tủ, máy_g..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4033,4033,"đường phạm văn đồng, phường đông ngạc, bắc từ...",0981959535,NaN,2.0,NaN,NaN,83,NaN,dự án sunshine city,NaN,NaN,NaN,NaN,NaN,9.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,chúng tôi chuyên cho thuê các căn hộ tại chung...,"[['sunshine cbắc từ_l', 'PJ'], ['2,3,4,5 phòng..."
4034,4034,"đường lê đức thọ, phường mỹ đình 2, nam từ li...",0348983388,NaN,3.0,NaN,NaN,94,NaN,dự án the zei mỹ đình,NaN,NaN,NaN,NaN,NaN,12.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,bên em đang quản lý nhiều căn hộ tại dự án the...,"[['căn_hộ', 'ID'], ['2n m2', 'ROOM'], ['cơ_bản..."
4035,4035,"đường hồ tùng mậu, phường phú diễn, bắc từ li...",0982848648,NaN,2.0,NaN,NaN,78,NaN,dự án goldmark city,NaN,NaN,NaN,NaN,NaN,9.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,do không có nhu cầu sử dụng vợ chồng tôi cần c...,"[['căn_hộ chung_cư cao_cấp', 'APART'], ['toà c..."
4036,4036,"đường lạc long quân, phường xuân la, tây hồ, ...",0976944818,NaN,3.0,NaN,NaN,114,NaN,dự án ciputra hà nội,NaN,NaN,NaN,NaN,NaN,15.0,https://batdongsan.com.vn/cho-thue-can-ho-chun...,hiện tại tôi có căn hộ tại tòa the link trong ...,"[['toà the link trong khu đô_thị ciputra.', 'P..."


In [41]:
filled_bds_df = fill_empty_data(converted_df=bds_df)

In [42]:
filled_bds_df

,Unnamed: 0,LOCATION,PHONE_NUMBER,NEWS_TYPE,BEDROOM_NUMBER,BATH_ROOM_NUMBER,LIVING_ROOM,AREA,APARTMENT_TYPE,PROJECT,INVESTOR,FLOOR,FURNITURE_TYPE,FURNITURE,CONVENIENT,PRICE,URL,RAW_DESCRIPTION,ENTITIES
0,0,"đường đại lộ thăng long, phường đại mỗ, nam t...",0984521346,NaN,1.0,NaN,NaN,35,chung_cư,dự án vinhomes smart city,NaN,NaN,full,[],"[vườn nhật, bệnh_viện trường, trung_tâm thương...",4,https://batdongsan.com.vn/cho-thue-can-ho-chun...,"* cam kết:\n- giá rẻ nhất, dịch vụ tốt nhất.\n...","[['đầy_đủ', 'FUR-TYPE'], ['cơ_bản', 'FUR-TYPE'..."
1,1,"phường tây mỗ, nam từ liêm, hà nội",0933722886,NaN,2.0,NaN,NaN,70,NaN,dự án vinhomes smart city đại mỗ,NaN,NaN,NaN,[],[],8,https://batdongsan.com.vn/cho-thue-can-ho-chun...,với kinh nghiệm 5 năm cho thuê nhà và đội ngũ ...,"[['<s>', 'AREA'], ['</s>', 'AREA'], ['<s>', 'A..."
2,2,"phường tây mỗ, nam từ liêm, hà nội",0833451111,NaN,1.0,1.0,NaN,35,studio,dự án vinhomes smart city đại mỗ,NaN,NaN,full,[],"[trung_tâm hội_nghị quốc_gia, miễn_phí tiện_íc...",4,https://batdongsan.com.vn/cho-thue-can-ho-chun...,"với sự uy tín, chu đáo và nhiệt tình của đội n...","[['cc vinhomes smart city', 'PJ'], ['1pn', 'RO..."
3,3,"ngõ 27, đường cát linh, phường cát linh, đống ...",0902469188,NaN,3.0,1.0,NaN,100,NaN,None,NaN,NaN,full,"[tủ_lạmáy_giặt, điều_hoà, nóng lạnh, giường t...",[],12,https://batdongsan.com.vn/cho-thue-can-ho-chun...,căn hộ đầu hồi 100m2 tầng 3 - tại ngõ 27 cát l...,"[['100m2', 'AREA'], ['ngõ 27 cát linh - đống_đ..."
4,4,"đường trần duy hưng, phường trung hòa, cầu gi...",0944010255,NaN,1.0,NaN,NaN,38,studio,dự án vinhomes d'capitale,NaN,NaN,full,"[tv, giường, tủ, máy_giặt, bàn ăn]","[tttm vincom., hầm để xe <unk> rã, bể_bơi. gym...",8,https://batdongsan.com.vn/cho-thue-can-ho-chun...,lh xem thực tế căn hộ 24/7:\n0944010***\n.\n\n...,"[['full', 'FUR-TYPE'], ['tv, giường, tủ, máy_g..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4033,4033,"đường phạm văn đồng, phường đông ngạc, bắc từ...",0981959535,NaN,2.0,NaN,NaN,83,NaN,dự án sunshine city,NaN,NaN,cơ_bản,[],"[trường maple bear, mầm_non, trường tiểu_học, ...",9,https://batdongsan.com.vn/cho-thue-can-ho-chun...,chúng tôi chuyên cho thuê các căn hộ tại chung...,"[['sunshine cbắc từ_l', 'PJ'], ['2,3,4,5 phòng..."
4034,4034,"đường lê đức thọ, phường mỹ đình 2, nam từ li...",0348983388,NaN,3.0,NaN,NaN,94,NaN,dự án the zei mỹ đình,NaN,NaN,cơ_bản,"[điều_hoà, nóng lạnh, bếp, rèmng]","[trung_tâm, hồ điều_hoà]",12,https://batdongsan.com.vn/cho-thue-can-ho-chun...,bên em đang quản lý nhiều căn hộ tại dự án the...,"[['căn_hộ', 'ID'], ['2n m2', 'ROOM'], ['cơ_bản..."
4035,4035,"đường hồ tùng mậu, phường phú diễn, bắc từ li...",0982848648,NaN,2.0,NaN,NaN,78,cao_cấp chung_cư,dự án goldmark city,NaN,NaN,NaN,"[thiết_bị vệ<unk>, hệ_thống đèn chiếu, nóng lạ...",[],9,https://batdongsan.com.vn/cho-thue-can-ho-chun...,do không có nhu cầu sử dụng vợ chồng tôi cần c...,"[['căn_hộ chung_cư cao_cấp', 'APART'], ['toà c..."
4036,4036,"đường lạc long quân, phường xuân la, tây hồ, ...",0976944818,NaN,3.0,2.0,NaN,114,NaN,dự án ciputra hà nội,NaN,NaN,full,[],[],15,https://batdongsan.com.vn/cho-thue-can-ho-chun...,hiện tại tôi có căn hộ tại tòa the link trong ...,"[['toà the link trong khu đô_thị ciputra.', 'P..."


In [43]:
filled_bds_df.to_csv('filled_bds.csv', index=False)

In [16]:
pd.isna(bds_df.iloc[3]['PROJECT'])

True

In [68]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="http")
add = "10 Phố Láng Hạ, Lang Ha, Đống Đa, Hanoi"

loc = geolocator.geocode(add)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

latitude is :- 21.0131278 
longtitude is:- 105.8126081
